# Лабораторная работа № 1
## Студент: Спиридонов К.А. М8О-107М-23

### Задачи:
Градиентный спуск и его модификации
   - Выбрать [тестовые функции оптимизации](https://ru.wikipedia.org/wiki/Тестовые_функции_для_оптимизации) (2 шт)
   - Запрограммировать собственнуб реализацию классического градиентного спуска
   - Запрограммировать пайлайн тестирования алгоритма оптимизации
     - Визуализации функции и точки оптимума
     - Вычисление погрешности найденного решения в сравнение с аналитическим для нескольких запусков
     - Визуализации точки найденного решения (можно добавить анимацию на плюс балл)
   - Запрограммировать метод вычисления градиента
     - Передача функции градиента от пользователя
     - Символьное вычисление градиента (например с помощью [sympy](https://www.sympy.org/en/index.html)) (на доп балл)
     - Численная аппроксимация градиента (на доп балл)
   - Запрограммировать одну моментную модификацию и протестировать ее
   - Запрограммировать одну адаптивную модификацию и протестировать ее
   - Запрограммировать метод эфолюции темпа обучения и/или метод выбора начального приближения и протестировать их

In [1]:
from matplotlib import pyplot as plt
import numpy as np
from numpy import linalg as la
import matplotlib.animation as animation
from IPython.display import HTML
import autograd
import math

In [2]:
class ParametersFunction:
    def __init__(self, name: str, domain: np.ndarray, glob_min: np.ndarray,
                 fun: callable, grad: callable = None) -> None:

        """ Инициализация параметров функции

        Args:
            domain (np.ndarray): Область определения функции
            glob_min (np.ndarray): Глобальный минимум
            fun (callable): Функция
        """
        self.name = name
        self.domain = domain
        self.glob_min = glob_min
        self.fun = fun
        self.grad = grad

In [3]:
Sphere = ParametersFunction(
    name = "Sphere",
    fun = lambda p: p[0]*p[0] + p[1]*p[1],
    grad = lambda p: np.array([2 * p[0], 2 * p[1]]),
    domain = np.array([[-3., -3.], [3., 3.]]),
    glob_min = np.array([0., 0., 0.])
    )

In [4]:
Booth = ParametersFunction(
    name = "Booth",
    fun = lambda p: (p[0] + 2 * p[1] - 7) ** 2 + (2 * p[0] + p[1] - 5) ** 2,
    grad = lambda p: np.array([10 * p[0] + 8 * p[1] - 34, 10 * p[1] + 8 * p[0] - 38]),
    domain = np.array([[-3., -3.], [7., 7.]]),
    glob_min = np.array([1., 3., 0.])
)

In [5]:
def numerical_grad(fun: callable, point: np.ndarray, dt: float = 0.00001) -> np.array:

    """Вычисляет градиент функции в точке

    Args:
        fun (callable): Функция искуственного ландшавта
        point (np.ndarray): Точка (массив параметров)
        dt (float, optional): Изменение аргумента. Defaults to 0.00001.

    Returns:
        np.array: Возвращает значение градиента в указанной точке
    """

    dxdt = (fun(point + np.array([dt, 0])) - fun(point)) / dt
    dydt = (fun(point + np.array([0, dt])) - fun(point)) / dt
    return np.array([dxdt, dydt])

In [6]:
def Gradient_Descent(function_info: ParametersFunction, start_point: np.ndarray,
                     max_iter: int = 64, lr: float = 0.1, delta: float = 0.001,
                     method: str = 'handle') -> np.array:

    """Классический градиентный спуск

    Args:
        function_info (ParametersFunction): Функция, у которой вычисляем градиент
        start_point (np.ndarray): Начальная точка
        max_iter (int, optional): Ограничение по кол-ву итераций. Defaults to 64.
        lr (float, optional): Шаг обучения. Defaults to 0.1.
        delta (float, optional): Радиус сходимости. Defaults to 0.001.
        method (str, optional): Способ, которым будет вычисляться градиент. Defaults to 'handle'

    Returns:
        np.array: История градиентного спуска
    """

    fun = function_info.fun
    glob_min = function_info.glob_min

    params = start_point.copy()
    history = [np.array([params[0], params[1], fun(params)])]

    symbolic_grad = 0
    if method == 'symbolic':
        symbolic_grad = autograd.grad(fun)

    step = 0
    while (step < max_iter and la.norm(history[-1] - glob_min) > delta):

        if method == 'handle':
            params = params - lr * function_info.grad(params)
        elif method == 'numerically':
            params = params - lr * numerical_grad(fun, params)
        elif method == 'symbolic':
            params = params - lr * symbolic_grad(params)


        history.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return np.array(history)

In [7]:
def Gradient_Descent_Momentum(function_info: ParametersFunction, start_point: np.ndarray,
                     max_iter: int = 64, lr: float = 0.1, beta: float = 0.1,
                     delta: float = 0.001, method = 'handle') -> np.array:

    """Градиентный спуск с использованием момементов

    Args:
        function_info (ParametersFunction): Функция, у которой вычисляем градиент
        start_point (np.ndarray): Начальная точка
        max_iter (int, optional): Ограничение по кол-ву итераций. Defaults to 64.
        lr (float, optional): Шаг обучения. Defaults to 0.1.
        beta (float, optional): Шаг обучения. Defaults to 0.1.
        delta (float, optional): Радиус сходимости. Defaults to 0.001.
        method (str, optional): Способ, которым будет вычисляться градиент. Defaults to 'handle'

    Returns:
        np.array: История градиентного спуска
    """

    fun = function_info.fun
    glob_min = function_info.glob_min

    # Рассчитываем начальный набор параметров
    params = start_point.copy()
    history = [np.array([params[0], params[1], fun(params)])]

    symbolic_grad = 0
    if method == 'symbolic':
        symbolic_grad = autograd.grad(fun)

    step = 0
    v_current = np.array([0, 0])
    while (step < max_iter and la.norm(history[-1] - glob_min) > delta):
        if method == 'handle':
            v_current = beta * v_current - lr * function_info.grad(params)
        elif method == 'numerically':
            v_current = beta * v_current - lr * numerical_grad(fun, params)
        elif method == 'symbolic':
            v_current = beta * v_current - lr * symbolic_grad(params)

        params = params + v_current

        history.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return np.array(history)

In [8]:
def Gradient_Descent_Momentum_Nesterov(function_info: ParametersFunction, start_point: np.ndarray,
                     max_iter: int = 64, lr: float = 0.1, beta: float = 0.1,
                     delta: float = 0.001, method = 'handle') -> np.array:

    """Градиентный спуск с использованием момементов

    Args:
        function_info (ParametersFunction): Функция, у которой вычисляем градиент
        start_point (np.ndarray): Начальная точка
        max_iter (int, optional): Ограничение по кол-ву итераций. Defaults to 64.
        lr (float, optional): Шаг обучения. Defaults to 0.1.
        beta (float, optional): Шаг обучения. Defaults to 0.1.
        delta (float, optional): Радиус сходимости. Defaults to 0.001.
        method (str, optional): Способ, которым будет вычисляться градиент. Defaults to 'handle'

    Returns:
        np.array: История градиентного спуска
    """

    fun = function_info.fun
    glob_min = function_info.glob_min

    # Рассчитываем начальный набор параметров
    params = start_point.copy()
    historty = [np.array([params[0], params[1], fun(params)])]

    symbolic_grad = 0
    if method == 'symbolic':
        symbolic_grad = autograd.grad(fun)

    step = 0
    v_current = np.array([0, 0])
    while (step < max_iter and la.norm(historty[-1] - glob_min) > delta):
        if method == 'handle':
            v_current = beta * v_current - lr * function_info.grad(params + beta * v_current)
        elif method == 'numerically':
            v_current = beta * v_current - lr * numerical_grad(fun, params + beta * v_current)
        elif method == 'symbolic':
            v_current = beta * v_current - lr * symbolic_grad(params + beta * v_current)

        params = params + v_current

        historty.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return np.array(historty)

In [9]:
def Adam(function_info: ParametersFunction, start_point: np.ndarray,
                     max_iter: int = 64, lr: float = 0.1, beta_1: float = 0.9,
                     beta_2: float = 0.99, eps: float = 1e-8,
                     delta: float = 0.001, method = 'handle') -> np.array:

    """ADAptive Momentum

    Args:
        function_info (ParametersFunction): Функция, у которой вычисляем градиент
        start_point (np.ndarray): Начальная точка
        max_iter (int, optional): Ограничение по кол-ву итераций. Defaults to 64.
        lr (float, optional): Шаг обучения. Defaults to 0.1.
        beta_1 (float, optional): Шаг обучения. Defaults to 0.9.
        beta_2 (float, optional): Шаг обучения. Defaults to 0.99.
        eps (float, optional): Шаг обучения. Defaults to 1e-8.
        delta (float, optional): Радиус сходимости. Defaults to 0.001.
        method (str, optional): Способ, которым будет вычисляться градиент. Defaults to 'handle'

    Returns:
        np.array: История градиентного спуска
    """

    fun = function_info.fun
    glob_min = function_info.glob_min

    # Рассчитываем начальный набор параметров
    params = start_point.copy()
    history = [np.array([params[0], params[1], fun(params)])]

    symbolic_grad = 0
    if method == 'symbolic':
        symbolic_grad = autograd.grad(fun)

    step = 0
    v = np.array([0, 0])
    G = np.array([0, 0])
    m_t = np.array([0, 0])
    v_t = np.array([0, 0])
    v_t_r = 0
    m_t_r = 0
    while (step < max_iter and la.norm(history[-1] - glob_min) > delta):

        step += 1
        if method == 'handle':
            g_t = function_info.grad(params)
        elif method == 'numerically':
            g_t = numerical_grad(fun, params)
        elif method == 'symbolic':
            g_t = symbolic_grad(params)

        m_t = beta_1 * m_t + (1 - beta_1) * g_t
        v_t = beta_2 * v_t + (1 - beta_2) * g_t**2
        m_t_r = m_t / (1 - beta_1**step)
        v_t_r = v_t / (1 - beta_2**step)

        params = params - (lr * m_t_r) / (np.sqrt(v_t_r) + eps)

        history.append(np.array([params[0], params[1], fun(params)]))

    return np.array(history)

[Источник](https://arxiv.org/pdf/1412.6980.pdf) на алгоритм `Adam`

In [10]:
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.animation as animation

draw_legend = True

def update_lines(num, ax, history):
    global draw_legend
    line = ax.plot(history[:num, 0], history[:num, 1], history[:num, 2], '-o', c='black', label = 'Градиентный спуск', alpha = 0.7)
    if draw_legend:
        draw_legend = False
        ax.legend(loc="upper left")

    return line

def draw_result(function_info: ParametersFunction, history: np.array, title: str) -> None:

    """Визуализация градиентного спуска

    Args:
        function_info (callable): Исходная функция
        history (np.array): История градиентного спуска
        title (str): Содержание заголовка
    """

    fun = function_info.fun
    glob_min = function_info.glob_min
    domain = function_info.domain

    global draw_legend
    draw_legend = True

    fig = plt.figure(figsize = (10, 10))
    ax = plt.axes(projection = '3d')

    x = np.linspace(domain[0, 0], domain[1, 0], 100)
    y = np.linspace(domain[0, 1], domain[1, 1], 100)

    x_grid, y_grid = np.meshgrid(x, y)
    z_grid = fun(np.array([x_grid, y_grid]))

    ax.plot_surface(x_grid, y_grid, z_grid, cmap = 'plasma', alpha=0.5)
    ax.scatter3D(history[0, 0], history[0, 1], history[0, 2], s=100, c="white", lw=2, ec='black', marker = 'D', label="Начальная точка")
    ax.scatter3D(history[-1, 0], history[-1, 1], history[-1, 2], s=190, c="white", lw=2, ec='black', marker = 'X', label="Найденный минимум")
    ax.scatter3D(glob_min[0], glob_min[1], glob_min[2], s=150, c="red", lw=2, ec='black', marker = 'o', label="Глобальный минимум", alpha = 0.7)

    np.set_printoptions(formatter={'float_kind':"{:.2f}".format})
    print(f"Начальная точка:\t\t\t{history[0]}")
    np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
    print(f"Найденный минимум:\t\t\t{history[-1]}")
    print(f"Глобальный минимум:\t\t\t{glob_min}")
    print(f"Кол-во итераций:\t\t\t{len(history)}")
    np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
    print(f"Погрешность найденного решения:\t\t{glob_min - history[-1]}")
    fig.text(0.9, 0.1, s=f"Кол-во итераций: {len(history)}", horizontalalignment="right", fontsize = 12)

    ax.set_title(title, fontsize = 12, fontweight="bold",loc="left")
    ax.legend(loc="upper left")
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    num_steps = min(1000, len(history))
    ani = animation.FuncAnimation(
            fig, update_lines, num_steps, fargs=(ax, history), interval=50)

    return ani

plt.ioff()

In [11]:
path = Gradient_Descent(Sphere, start_point = np.array([5.0, 5.0]), max_iter = 100, delta = 0.001, method = 'handle')
anim = draw_result(Sphere, path, "Классический градиентный спуск")
plt.ioff()

Начальная точка:			[5.00 5.00 50.00]
Найденный минимум:			[6.65e-04 6.65e-04 8.83e-07]
Глобальный минимум:			[0.00e+00 0.00e+00 0.00e+00]
Кол-во итераций:			41
Погрешность найденного решения:		[-6.65e-04 -6.65e-04 -8.83e-07]


In [12]:
HTML(anim.to_html5_video())

In [13]:
path = Gradient_Descent(Booth, start_point = np.array([5., 5.]), method = 'handle')
anim = draw_result(Booth, path, "Классический градиентный спуск")
plt.ioff()

Начальная точка:			[5.00 5.00 164.00]
Найденный минимум:			[1.00e+00 3.00e+00 7.07e-06]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			39
Погрешность найденного решения:		[-8.31e-04 -4.15e-04 -7.07e-06]


In [14]:
HTML(anim.to_html5_video())

In [16]:
path = Gradient_Descent_Momentum(Booth, start_point = np.array([5., 5.]), method = 'symbolic')
anim = draw_result(Booth, path, "Моментный градиентный спуск")
plt.ioff()

Начальная точка:			[5.00 5.00 164.00]
Найденный минимум:			[1.00e+00 3.00e+00 9.75e-07]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			29
Погрешность найденного решения:		[-6.98e-04 6.98e-04 -9.75e-07]


In [17]:
HTML(anim.to_html5_video())

In [18]:
path = Gradient_Descent_Momentum_Nesterov(Sphere, start_point = np.array([5., 5.]), max_iter = 100, method = 'numerically')
anim = draw_result(Sphere, path, "Нестерова моментный радиентный спуск")
plt.ioff()

Начальная точка:			[5.00 5.00 50.00]
Найденный минимум:			[5.83e-04 5.83e-04 6.80e-07]
Глобальный минимум:			[0.00e+00 0.00e+00 0.00e+00]
Кол-во итераций:			37
Погрешность найденного решения:		[-5.83e-04 -5.83e-04 -6.80e-07]


In [19]:
HTML(anim.to_html5_video())

In [20]:
path = Gradient_Descent_Momentum_Nesterov(Booth, start_point = np.array([5., 5.]), beta = 0.01, max_iter = 100, method = 'handle')
anim = draw_result(Booth, path, "Нестерова моментный радиентный спуск")
plt.ioff()

Начальная точка:			[5.00 5.00 164.00]
Найденный минимум:			[1.00e+00 3.00e+00 7.13e-06]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			43
Погрешность найденного решения:		[-7.05e-04 -5.52e-04 -7.13e-06]


In [21]:
HTML(anim.to_html5_video())

In [22]:
path = Adam(Sphere, start_point = np.array([5., 5.]), max_iter = 300, method = 'numerically')
anim = draw_result(Sphere, path, "Adam")
plt.ioff()

Начальная точка:			[5.00 5.00 50.00]
Найденный минимум:			[-2.30e-04 -2.30e-04 1.05e-07]
Глобальный минимум:			[0.00e+00 0.00e+00 0.00e+00]
Кол-во итераций:			81
Погрешность найденного решения:		[2.30e-04 2.30e-04 -1.05e-07]


In [23]:
HTML(anim.to_html5_video())

In [24]:
path = Adam(Booth, start_point = np.array([5., 5.]), max_iter = 300, method = 'handle')
anim = draw_result(Booth, path, "Adam")
plt.ioff()

Начальная точка:			[5.00 5.00 164.00]
Найденный минимум:			[1.00e+00 3.00e+00 9.52e-07]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			259
Погрешность найденного решения:		[-6.92e-04 6.87e-04 -9.52e-07]


In [25]:
HTML(anim.to_html5_video())

In [26]:
path = Adam(Booth, start_point = np.array([5., 5.]), max_iter = 300, method = 'numerically')
anim = draw_result(Booth, path, "Adam")
plt.ioff()

Начальная точка:			[5.00 5.00 164.00]
Найденный минимум:			[1.00e+00 3.00e+00 9.52e-07]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			259
Погрешность найденного решения:		[-6.90e-04 6.90e-04 -9.52e-07]


In [27]:
path = Adam(Booth, start_point = np.array([5., 5.]), max_iter = 300, method = 'symbolic')
anim = draw_result(Booth, path, "Adam")
plt.ioff()

Начальная точка:			[5.00 5.00 164.00]
Найденный минимум:			[1.00e+00 3.00e+00 9.52e-07]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			259
Погрешность найденного решения:		[-6.92e-04 6.87e-04 -9.52e-07]
